In [206]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [207]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [208]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [209]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [210]:
# TASK 1
players_180_190 = player_data.loc[(player_data.height >= 180) & (player_data.height <= 190)].shape[0]

In [211]:
assert(isinstance(players_180_190, int))

In [212]:
# TASK 2
import re
players_1980 = player_data.loc[player_data.birthday.str.match(r'^1980-.*') == True].shape[0]

In [213]:
assert(isinstance(players_1980, int))

In [214]:
# TASK 3
highest_players = list(player_data.sort_values(by=['weight', 'player_name'],\
                                               ascending = [False, True], \
                                               ignore_index=True).loc[:9, 'player_name'])

In [215]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

In [216]:
# TASK 4
years_born_players = []
player_data.birthday = pd.to_datetime(player_data['birthday'])
date_80_90 = player_data.birthday.dt.year.\
             loc[(player_data.birthday.dt.year <= 1990) & (player_data.birthday.dt.year >= 1980)].\
             value_counts().sort_index().array
df_date_80_90 = pd.DataFrame(date_80_90, range(1980,1991,1))
for i, row in df_date_80_90.iterrows():
    years_born_players.append((i, int(row[0])))    

In [217]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

In [218]:
# TASK 5
adriano_mean, adriano_std = player_data.loc[player_data.player_name.str.match(r'^Adriano.*') == 1].height.mean(),\
                            player_data.loc[player_data.player_name.str.match(r'^Adriano.*') == 1].height.std()

In [219]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

In [220]:
# TASK 6
dow_with_min_players_born = player_data['birthday'].dt.day_name().value_counts('Day').sort_values().index[0]

In [221]:
assert(isinstance(dow_with_min_players_born, str))

In [222]:
# TASK 7
league = pd.read_sql("SELECT * FROM League;", db)
matche = pd.read_sql("SELECT * FROM Match;", db)
leag_id = matche.value_counts('league_id').sort_values(ascending = False)
leag_id_max_rep = int(leag_id.value_counts(sort=False).values[0])
leags_most_matches = leag_id.index[0:leag_id_max_rep]
league_most_matches = league.name.loc[league.id.isin(leags_most_matches)].sort_values().values[0]

In [223]:
assert(isinstance(league_most_matches, str))

In [224]:
# TASK 8
player_id = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',\
             'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',\
             'home_player_9', 'home_player_10', 'home_player_11',\
             'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',\
             'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',\
             'away_player_9', 'away_player_10', 'away_player_11']
id_df = pd.DataFrame({'id': matche.loc[:][player_id[0]]})
for i in player_id[1:]:
    id_df1 = pd.DataFrame({'id': matche.loc[:][i]})
    id_df = pd.concat([id_df, id_df1], ignore_index=True)
id_player = id_df.value_counts().index[0]
max_matches_player = list(player_data.loc[player_data.player_api_id == id_player]['player_name'])[0]

In [225]:
assert(isinstance(max_matches_player, str))

In [226]:
# TASK 9
player_attr = pd.read_sql("SELECT * FROM Player_attributes;", db)
player_char = []
player_char_exept = ['id', 'player_fifa_api_id', 'player_api_id', 'date',\
                     'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
for char in list(player_attr.columns):
    if char not in player_char_exept:
        player_char.append(char)
df_attr = pd.DataFrame(player_attr.loc[:][player_char]) 
corr = df_attr.corr(method='pearson')
dict = {}
array_max = [0]
top_correlated_features = []
for i in range(0, int(corr.size**0.5)):
    for j in range(0, int(corr.size**0.5)):
        if i >= j:
            corr.iloc[i, j] = 0
        elif pd.notna(corr.iloc[i, j]):
            dict[corr.iloc[i, j]] = (str(corr.index[i]), str(corr.index[j]))
            array_max.append(corr.iloc[i, j])
            
array_max.sort(reverse = True)            
for el in array_max[:5]:
    top_correlated_features.append(dict[el])

In [227]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

In [228]:
#TASK 10
import numpy as np
neymar_id = int(player_data.player_api_id.loc[player_data.player_name.str.count(r'Neymar') == 1])
neymar_vect = player_attr.loc[player_attr.player_api_id == neymar_id].\
                         sort_values(by='date', ascending = False, ignore_index = True).\
                         loc[0, player_char]
player_char.append('player_api_id')
player_char.append('date')
df_evk_s = pd.DataFrame(player_attr.loc[:][player_char]).\
                     sort_values(by=['player_api_id', 'date'], ignore_index = True, ascending = [True, False])
count_uniq = list(df_evk_s.value_counts('player_api_id',sort=False).values)
array = [0]
for el in count_uniq:
    array.append(array[-1]+el)
evk_vect_full = df_evk_s.loc[array[0:-1],:].set_index('player_api_id').drop('date', axis=1)
evk_vect = (evk_vect_full - neymar_vect)**2
evk = evk_vect.apply(np.sum, axis=1).apply(np.sqrt)
most_similar_id = evk.apply(lambda r: r-evk.loc[neymar_id]).abs().sort_values().index[1:6].to_list()
neymar_similarities = []
for el in most_similar_id:
    neymar_similarities.append(list(player_data.loc[player_data.player_api_id == el].player_name)[0])

In [229]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

In [230]:
# TASK 11
team_data = pd.read_sql("SELECT * FROM Team;", db)
team_attr = pd.read_sql("SELECT * FROM Team_attributes;", db)
dort_id = int(team_data.loc[team_data.team_long_name == 'Borussia Dortmund']['team_api_id'].values[0])
dort_sh = team_data.loc[team_data.team_long_name == 'Borussia Dortmund']['team_short_name'].values[0]
leag_id = int(league.loc[league.name == 'Germany 1. Bundesliga']['id'].values[0])
borussia_bundesliga_2008_2009_matches = matche.loc[(matche.season == '2008/2009') & \
                                                   (matche.league_id == leag_id) & \
                                                   (matche['home_team_api_id'] == dort_id)].shape[0]

In [231]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

In [232]:
# TASK 12
team_most_matches_bundesliga_2008_2009_arr = matche.loc[(matche.season == '2008/2009') &\
                                                        (matche.league_id == leag_id)]\
                                                       [['home_team_api_id' ,'away_team_api_id']]

team_df = pd.concat([team_most_matches_bundesliga_2008_2009_arr.home_team_api_id, \
                     team_most_matches_bundesliga_2008_2009_arr.away_team_api_id], ignore_index=True)
team_most_matches_bundesliga_2008_2009 = int(team_df.value_counts(ascending = False).values[0])                   

In [233]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

In [234]:
#TASK 13
ars_id = int(team_data.loc[team_data.team_long_name == 'Arsenal']['team_api_id'].values[0])

home_won = matche.loc[(matche.season == '2015/2016') & (matche.home_team_api_id == ars_id)].\
       loc[matche.home_team_goal > matche.away_team_goal].shape[0]

away_won = matche.loc[(matche.season == '2015/2016') & (matche.away_team_api_id == ars_id)].\
       loc[matche.home_team_goal < matche.away_team_goal].shape[0]

arsenal_won_matches_2015_2016 = away_won + home_won


In [235]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

In [236]:
#TASK 14
ser_home = matche.loc[(matche.season == '2015/2016')].value_counts('home_team_api_id', sort = False)
ser_away = matche.loc[(matche.season == '2015/2016')].value_counts('away_team_api_id', sort = False)
won_home = []
for el in list(ser_home.index):
    ser_home_won = matche.loc[(matche.season == '2015/2016') & (matche.home_team_api_id == el)].\
    loc[matche.home_team_goal > matche.away_team_goal].shape[0]
    won_home.append(ser_home_won)
won_away = []
for el in list(ser_home.index):
    ser_away_won = matche.loc[(matche.season == '2015/2016') & (matche.away_team_api_id == el)].\
    loc[matche.home_team_goal < matche.away_team_goal].shape[0]
    won_away.append(ser_away_won)
team_rate = pd.DataFrame({'home_team':ser_home, 'away_team':ser_away, \
                          'home_won':won_home, 'away_won':won_away}, index=ser_home.index)
ser_games = team_rate.loc[:,['home_team', 'away_team']].sum(axis = 1)
ser_wons = team_rate.loc[:,['home_won', 'away_won']].sum(axis = 1)
rate = []
for el in ser_home.index:
    rate.append(ser_wons[el]/ser_games[el])
team_highest_winrate_2015_2016 = list(team_data.loc[team_data.team_api_id == int(pd.Series(rate, index=ser_home.index).\
                                                sort_values(ascending = False).index[0])]['team_long_name'])[0]

In [237]:
assert(isinstance(team_highest_winrate_2015_2016, str))

In [238]:
# TASK 15
eng_leag_id = int(league.loc[league.name == 'England Premier League']['id'].values[0])

team_eng = matche.loc[(matche.season == '2010/2011') & (matche.league_id == eng_leag_id)]\
                     [['date','away_team_api_id','home_team_api_id']]
team_eng_uni = pd.concat([team_eng.rename(columns={'home_team_api_id': 'team_api_id'}).loc[:,['date','team_api_id']],\
                          team_eng.rename(columns={'away_team_api_id': 'team_api_id'}).loc[:,['date','team_api_id']]],\
                          ignore_index=True)\
                          .sort_values(by=['team_api_id', 'date'], ascending = [True, False], ignore_index = True)
team_eng_uni.date = pd.to_datetime(team_eng_uni.date)

arr_ammount = list(team_eng_uni.value_counts('team_api_id', sort = False).values)
arr_ind = [0]
for el in arr_ammount:
    arr_ind.append(arr_ind[-1]+el)
arr_id = list(team_eng_uni.value_counts('team_api_id', sort = False).index)
time_dif = [0]
for i in range(0,len(arr_id)): # i команда
    for j in range(arr_ind[i],arr_ind[i+1]-1): # диапазон индкесов для команды
        time_dif_temp = (team_eng_uni.loc[j,'date']-team_eng_uni.loc[j+1,'date']).days
        if time_dif_temp > time_dif[-1]:
            time_dif.append(time_dif_temp)
               
highest_gap_england_2010_2011 = time_dif[-1]

In [239]:
assert(isinstance(highest_gap_england_2010_2011, int))

In [240]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")